# How to optimize target VM costs when moving them to Azure using an SMT solver

## Scenario

[Last time](/vmOptimization-basics.ipynb) we optimized the total VM cost in the Azure Cloud using minizinc. This time we want to avoid installing additional software and use a solver directly, from .NET. For this experiment, we want to use [or-tools](https://developers.google.com/optimization/introduction/overview) - the SMT solver from Google.


> OR-Tools is open source software for combinatorial optimization, which seeks to find the best solution to a problem out of a very large set of possible solutions. Here are some examples of problems that OR-Tools solves:
>
> - Vehicle routing: Find optimal routes for vehicle fleets that pick up and deliver packages given constraints (e.g., "this truck can't hold more than 20,000 pounds" or "all deliveries must be made within a two-hour window").
> - Scheduling: Find the optimal schedule for a complex set of tasks, some of which need to be performed before others, on a fixed set of machines, or other resources.
> - Bin packing: Pack as many objects of various sizes as possible into a fixed number of bins with maximum capacities.

One of the things here is that or-tools has a .NET interface, so we can use it in Powershell without having to install anything in addition

> There is a small issue, though. The solver is C, I guess. The .NET interface is just a wrapper. For it to work it needs a [vc runtime](https://docs.microsoft.com/en-us/cpp/windows/latest-supported-vc-redist?view=msvc-170). But the Powershell module we are going to use already includes required libraries

To simplify stuff, I made a simple PowerShell module [AzureVmCalc](https://github.com/eosfor/AzureVmCalc). It contains everything needed to start working.

> At the moment, the module is still alpha, but it is working and can be used

Installing it

In [ ]:
Install-Module AzureVmCalc -AllowPrerelease -Scope CurrentUser

Importing

In [ ]:
Import-Module AzureVmCalc

Prepare the data

In [ ]:
$sourceVMs = Import-Csv ".\vm-optimization-ortools\vmdata.csv"
$sourceVMs  | % { $_.cpu = [int]$_.cpu; $_.ram = [int]$_.ram; $_.datadisk = [int]$_.datadisk; }

$targetSizes = import-csv ".\vm-optimization-ortools\vmCostACUData.csv"

The `vmdata.csv` file contains information about the "source" server sizes. The `vmCostACUData` contains data about VM sizes and their price from an Azure region

Now we can start the commandlet

In [ ]:
$x = Start-OrToolsModelCalculation -SourceVM $sourceVMs -TargetVM $targetSizes

The commandlet takes the list of source servers, the list of target sizes with their attributes and returns an array. Under the hood, processing goes in two stages. First, the model tries to maximize the total performance of the whole set of VMs, by using the ACU metric, supplied by Microsoft. Then it pins the performance objective and tries to minimize the total cost, keeping the performance objective intact. In the end, the output contains the results of both calculations

In [ ]:
$x


VmMappingResult
---------------
{Standard_E96-48as_v4, Standard_E96-48as_v4, Standard_E96-48as_v4, Standard_E96-48as_v4…}
{Standard_E4-2as_v4, Standard_E4-2as_v4, Standard_E4-2as_v4, Standard_E8-4as_v4…}



In [ ]:
$view = $x[1].VmMappingResult | select vmid, cpu, ram, name, vCPUs, MemoryGB, vCPUsPerCore, ACUs, retailPrice | ConvertTo-Html -Fragment
[Microsoft.DotNet.Interactive.Kernel]::HTML($view) | Out-Display

vmid,cpu,ram,Name,vCPUs,MemoryGB,vCPUsPerCore,ACUs,retailPrice
vmN1,2,12,Standard_E4-2as_v4,4,32,2,230,0.436
vmN2,2,24,Standard_E4-2as_v4,4,32,2,230,0.436
vmN3,2,4,Standard_E4-2as_v4,4,32,2,230,0.436
vmN4,6,23,Standard_E8-4as_v4,8,64,2,230,0.872
vmN5,3,16,Standard_E4-2as_v4,4,32,2,230,0.436
vmN6,2,32,Standard_E4-2as_v4,4,32,2,230,0.436
vmN7,2,7,Standard_E4-2as_v4,4,32,2,230,0.436
vmN8,3,32,Standard_E4-2as_v4,4,32,2,230,0.436
vmN9,2,8,Standard_E4-2as_v4,4,32,2,230,0.436
vmN10,1,40,Standard_E8-4as_v4,8,64,2,230,0.872
